# Создаем CSV-файл с данными о топ-1 и топ-500 компьютерах за все время.

На сайте https://top500.org нет сводной статистики за все время. Есть только данные за июнь и ноябрь месяц каждого года начиная с 1993. Спарсим информацию из этих файлов и объединим ее в один.

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt

Скачать файлы можно только автризовавшись на сайте. Чтобы не писать парсер авторизующийся на сайте, сделаем следующее: 
* авторизуемся на сайте в браузере по умолчанию
* сгенерируем ссылки на все файлы с рейтингами суперкомпьютеров
* Прокликаем по ссылкам и сохраним все файлы в папке data

In [ ]:
for year in range(1993, 2021):
    for mounth in ['06', '11']:
        wget_str = f'https://top500.org/lists/top500/{year}/{mounth}/download/TOP500_{year}{mounth}.xls'
        if year == 2020 and mounth == '06':
            print(wget_str + 'x')
        elif year == 2020 and mounth == '11':
            # закончились данные
            break
        else:
            print(wget_str)

In [ ]:
data_list = sorted(os.listdir('data'))
# data_list[:5], data_list[-5:]

Соберем исторические данные о топ-1, топ-500 и суммарной мощности в отдельные списки.

Файлы имеют разный индекс у заголовка и отличаются названияим целевого столбца, поэтому обрабатываем файлы в четырех циклах.

In [ ]:
data_1 = []
data_500 = []
data_sum = []

for i in range(30):
    df = pd.read_excel(os.path.join('data', data_list[i]), header=1, index_col=0)
    data_1.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['RMax'].iloc[0]))
    data_500.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['RMax'].iloc[-1]))
    data_sum.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['RMax'].sum()))
    
for i in range(30, 37):
    df = pd.read_excel(os.path.join('data', data_list[i]), header=0, index_col=0)
    data_1.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['RMax'].iloc[0]))
    data_500.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['RMax'].iloc[-1]))
    data_sum.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['RMax'].sum()))
    
for i in range(37, 48):
    df = pd.read_excel(os.path.join('data', data_list[i]), header=0, index_col=0)
    data_1.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['Rmax'].iloc[0]))
    data_500.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['Rmax'].iloc[-1]))
    data_sum.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['Rmax'].sum()))
    
for i in range(48, 55):
    df = pd.read_excel(os.path.join('data', data_list[i]), header=0, index_col=0)
    data_1.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['Rmax [TFlop/s]'].iloc[0]*1000))
    data_500.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['Rmax [TFlop/s]'].iloc[-1]*1000))
    data_sum.append((f'{data_list[i][11:13]}/{data_list[i][7:11]}', df['Rmax [TFlop/s]'].sum()*1000))
    

Собираем данные в pandas DataFrame и созраняем с CSV формате.

In [ ]:
parse_data = {
    'data': [i[0] for i in data_1],
    '#1 GFlop/s': [i[1] for i in data_1],
    '#500 GFlop/s': [i[1] for i in data_500],
    'Sum GFlop/s': [i[1] for i in data_sum],
}

In [ ]:
df = pd.DataFrame(parse_data)
df.head()

In [ ]:
df.to_csv('data_top500_hpc.csv')